In [1]:
import csv
import sqlite3

def lire_fichier():
    """
    Lecture du fichiers stops.txt du dossier BIBUS
    Renvoie les différentes lignes sous forme de liste
    """
    liste=[]
    with open('BIBUS/stops.txt') as f:
        lire=csv.reader(f, delimiter=',')
        for ligne in lire:
            liste.append(ligne)
    return liste

def execode():
    """
    Exécution de code SQL
    Ici, la méthode crée la table arrets si elle n'existe pas
    """
    connexion = sqlite3.connect('transports.db')
    curseur=connexion.cursor()
    curseur.executescript(
    """
    CREATE TABLE IF NOT EXISTS arrets(
    stop_id TEXT PRIMARY KEY NOT NULL,
    stop_name TEXT,
    stop_lat REAL,
    stop_lon REAL,
    wheelchair_boarding INT
    )
    """)
    connexion.commit() #Sauvegarde des changements
    connexion.close() #Fermeture de la DB
    
def connect(db):
    """
    Connexion à la DB passée en paramètre
    Renvoie la connexion et le curseur, utile pour les manipulations
    """
    connexion = sqlite3.connect(f'{db}.db')
    curseur=connexion.cursor()
    return connexion, curseur

def inputall(liste):
    """
    Ajout de tous les éléments à la DB
    A n'exécuter qu'une fois
    """
    connexion, curseur=connect("transports")
    liste.pop(0) #Suppression du 1er élément, correspondant aux attributs
    for elt in liste:
        curseur.execute(
            """INSERT INTO arrets (stop_id, stop_name, stop_lat, stop_lon, wheelchair_boarding) 
            VALUES (?,?,?,?,?)""",elt)
        connexion.commit()
    connexion.close()

def selectall():
    """
    Selection de l'entièreté de la DB
    """
    connexion, curseur=connect("transports")
    curseur.execute("SELECT * FROM arrets")
    rows = curseur.fetchall()
    connexion.close()    
    return rows

def GPS(arret):
    """
    Renvoie les coordonnées GPS lat et long de l'arret passé en paramètre
    """
    connexion, curseur=connect("transports")
    curseur.execute("SELECT stop_lat, stop_lon FROM arrets WHERE stop_name=?", [arret])
    rows = curseur.fetchall()
    connexion.close()
    return rows

def accesPMR():
    """
    Renvoie le nombre d'arret (count) possédant un accès PMR
    """
    connexion, curseur=connect("transports")
    curseur.execute("SELECT count(wheelchair_boarding) FROM arrets WHERE wheelchair_boarding=1")
    row = curseur.fetchone()
    connexion.close()
    return row[0]
        
    
#Programme principal
if __name__=='__main__':
    liste=lire_fichier()
    execode()
    #inputall(liste)
    selectall()
    print(GPS('Mirabeau'))
    print(accesPMR())

[(48.40994, -4.485379), (48.409554, -4.485374), (48.410255, -4.485921)]
763
